### 3.4 3층 신경망 구현하기

In [8]:
import numpy as np
import matplotlib.pylab as plt
X=np.array([1.0,0.5])
W1=np.array([[0.1,0.3,0.5],[0.2,.4,.6]])
B1=np.array([0.1,0.2,0.3])

In [3]:
print(W1.shape)

(2, 3)


In [4]:
print(X.shape)

(2,)


In [5]:
print(B1.shape)

(3,)


In [6]:
A1=np.dot(X,W1)+B1

은닉층에서의 가중치의 합(가중신호와 편향의 총합)을 a로 표기하고 활성화 함수 h()로 변환된 신호를 z로 표기

활성화 함수로 시그모이드 함수를 사용하기로 할때 구현은 아래와 같음

In [9]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

Z1=sigmoid(A1)

print(A1)

[0.3 0.7 1.1]


In [11]:
print(Z1)

[0.57444252 0.66818777 0.75026011]


In [12]:
# 1층에서 2층으로 가는 과정과 그 구현(p87)

W2=np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])
B2=np.array([0.1,0.2])

print(Z1.shape) #(3,)
print(W2.shape) #(3,2)
print(B2.shape) #(2,)

A2=np.dot(Z1,W2)+B2
Z2=sigmoid(A2)

(3,)
(3, 2)
(2,)


In [13]:
# 2층에서 출력층으로의 신호 전달
# 활성화 함수만 지금까지의 은닉층과 다름

def identity_function(x):
    return x

W3 = np.array([[0.1,0.3],[0.2,0.4]])
B3 = np.array([0.1,0.2])

A3=np.dot(Z2,W3)+B3
Y=identity_function(A3) #혹은 Y=A3

In [14]:
Y

array([0.31682708, 0.69627909])

##### 3.4.3 구현 정리

In [15]:
def init_network(): #가중치와 편향을 초기화 하고 이들을 딕셔너리 변수인 network에 저장
    
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5],[0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3],[0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    
    return network

#딕셔너리 변수인 network에는 각 층에 필요한 매개변수(가중치와 편향)를 저장

#forward()함수는 입력신호를 출력으로 변환하는 처리과정을 모두 구현

#why forward? 신호가 순방향(입->출)으로 전달됨(순전파)을 알리기 위함

def forward(network, x):
    
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = a3
    
    return y

In [16]:
network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


### 3.5 출력층 설계하기

신경망은 분류와 회귀 모두에 이용할 수 있지만, 둘 중 어떤 문제냐에 따라 출력층에서 사용하는 활성화 함수가 달라짐

일반적으로 **회귀**에는 **항등 함수**를, **분류**에는 **소프트맥스 함수**를 사용

##### 3.5.1 항등 함수와 소프트맥스 함수 구현하기

__항등함수__는 입력을 그대로 출력(입력과 출력이 항상 같음)-> 출력층에서 항등함수를 사용하면 입력신호가 그대로 출력신호가 됨

__소프트맥스 함수__는 출력층의 각 뉴런이 모든 입력 신호에서 영향을 받음

##### 3.5.2 소프트맥스 함수 구현 시 주의점

__overflow 문제__? 소프트맥스 함수는 지수함수를 사용하는데, 지수 함수는 쉽게 아주 큰 값을 내뱉음 <br>
큰 값끼리 나눗셈을 하면 결과 수치가 **불안정**해짐<br>
소프트맥스의 지수함수를 계산할 때 어떤 정수를 더해도 결과는 바뀌지 않음.<br>
C`에 어떤 값을 대입해도 상관없지만 오버플로우를 막을 목적으로 입력신호 중 최댓값을 이용하는 것이 일반적임

In [19]:
#나쁜 예제

a=np.array([1010,1000,990])
np.exp(a)/np.sum(np.exp(a))

<ipython-input-19-6b9416e1bd82>:4: RuntimeWarning: overflow encountered in exp
  np.exp(a)/np.sum(np.exp(a))
<ipython-input-19-6b9416e1bd82>:4: RuntimeWarning: invalid value encountered in true_divide
  np.exp(a)/np.sum(np.exp(a))


array([nan, nan, nan])

In [21]:
C=np.max(a)
a-C

array([  0, -10, -20])

In [23]:
np.exp(a-C)/np.sum(np.exp(a-C))

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

아무런 조치 없이 그냥 계산하면 nan이 출력되지만 입력 신호 중 최댓값을 빼주면 올바르게 계산할 수 있음

In [24]:
def softmax(a):
    
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로우 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

##### 3.5.3 소프트맥스 함수의 특징

- 소프트맥스 함수의 출력은 0에서 1.0 사이의 실수임<br>
- 소프트맥스 함수 출력의 총합은 1임 --> 이 성질 덕분에 소프트맥스 함수의 출력을 '확률'로 해석할 수 있음
- 주의! 소프트맥스 함수를 적용해도 각 원소의 대소관계는 변하지 않음

신경망을 이용한 분류에서는 일반적으로 가장 큰 출력을 내는 뉴런에 해당하는 클래스로만 인식하며, 소프트맥스 함수를 적용해도 출력이 가장 큰 뉴런의 위치는 달라지지 않음 (Q. 뭐가?)

결과적으로 신경망으로 분류할 때는 출력층의 소프트맥스 함수를 생략해도 됨. 